# **Assignment 1**: Computing the area of the Mandelbrot Set

Karolina Chlopicka, 15716546 <br>
Shania Sinha, 14379031 <br>
Salomé Poulain, 13955993

In [ ]:
# Libraries
import numpy as np
import matplotlib.pyplot as plt
from typing import List, Tuple, Callable
from scipy.stats import qmc  # For Latin Hypercube sampling
from joblib import Parallel, delayed

%matplotlib inline

## Problem 1: Mandelbrot Set

Creating and visualizing an example of the Mandelbrot set.

In [ ]:
BOUNDING_BOX_X = (-2.5, 1.5)
BOUNDING_BOX_Y = (-2, 2)
BOUND = 2
RESOLUTION_POINTS = 500
POWER = 2
MAX_ITER = 100 # Maximum number of iterations to check for convergence
SEED = 42

LITERARY_VALUE = 1.506591856 ####### idk

# COLOURS ??? maybe nice for the future 

In [ ]:
def build_mandelbrot(
    X: np.ndarray, 
    Y: np.ndarray, 
    bound: float = BOUND,
    power: int = POWER, 
    max_iter: int = MAX_ITER
) -> List[List[int]]:
    """
    Compute the Mandelbrot set for given x and y ranges.

    Parameters:
        X (np.ndarray): Array of x values (real parts of complex plane).
        Y (np.ndarray): Array of y values (imaginary parts of complex plane).
        bound (float): Threshold for divergence in the Mandelbrot set.
        power (int): The exponent for complex power in Mandelbrot iteration.
        max_iter (int): Maximum number of iterations to compute per point.

    Returns:
        List[List[int]]: 2D list where each element represents the iteration count 
                         before divergence, or 0 if the point belongs to the set.
    """
    mandelbrot_set = []
    X, Y = np.meshgrid(X, Y)
    C = X + 1j * Y  # Create a grid of complex numbers
    Z = np.zeros_like(C, dtype=complex)
    div_iter = np.zeros(C.shape, dtype=int)  # Track divergence iteration counts

    # Iteratively compute the Mandelbrot set
    for i in range(1, max_iter + 1):
        mask = np.abs(Z) < bound
        Z[mask] = Z[mask] ** power + C[mask]  # Update only non-diverging points
        div_iter[mask & (np.abs(Z) >= bound)] = i  # Record when they diverge

    mandelbrot_set = div_iter.tolist()  # Convert to a list of lists
    return mandelbrot_set

In [ ]:
# Setting parameters
all_x = np.linspace(BOUNDING_BOX_X[0], BOUNDING_BOX_X[1], RESOLUTION_POINTS)
all_y = np.linspace(BOUNDING_BOX_Y[0], BOUNDING_BOX_Y[1], RESOLUTION_POINTS)

colormap = 'magma'

mandelbrot_set = build_mandelbrot(all_x, all_y)

In [ ]:
# Plotting
ax = plt.axes()
ax.set_aspect('equal')
graph = ax.pcolormesh(all_x, all_y, mandelbrot_set, cmap = colormap, shading='auto')
plt.colorbar(graph)
plt.xlabel("Real-Axis")
plt.ylabel("Imaginary-Axis")
plt.title(f"Mandelbrot set for $z_{{new}} = z^{{power}} + c$")
plt.show()

## Problem 2: Investigate the convergence of $A_{i,s} \rightarrow A_M$

Investigating the area of the Mandelbrot set $A_M$ using Monte Carlo integration. $A_{i,s}$ denotes an estimate of an area, where $i$ refers to a number of iterations and $s$ refers to number of samples drawn.  

In [ ]:
def _check_points_mandelbrot(
    X: np.ndarray, Y: np.ndarray) -> np.ndarray:
    """
    Check if points in a grid are within the Mandelbrot set.

    Parameters:
        X (np.ndarray): Array of x-coordinates (real parts of complex numbers).
        Y (np.ndarray): Array of y-coordinates (imaginary parts of complex numbers).
        bound (float): Threshold for divergence in the Mandelbrot set (default is 2.0).
        max_iter (int): Maximum number of iterations for checking divergence (default is 100).

    Returns:
        np.ndarray: Boolean array of the same shape as X and Y, where True indicates
                    that the point is in the Mandelbrot set (does not diverge within the
                    iteration limit), and False indicates divergence.
    """
    # Create the complex plane grid from X and Y values
    C = X + 1j * Y
    Z = np.zeros_like(C, dtype=complex)  
    mask = np.ones(C.shape, dtype=bool)  

    # Iteratively compute the Mandelbrot set
    for _ in range(MAX_ITER):
        Z[mask] = Z[mask] ** POWER + C[mask] 
        mask &= np.abs(Z) < BOUND            

    return mask

In [ ]:
def monte_carlo_random(i: int, sample_size: int) -> np.ndarray:
    """
    Monte Carlo simulation to estimate the area of the Mandelbrot set.

    Parameters:
        i (int): Number of iterations.
        sample_size (int): Number of samples per iteration.
        bound (float): Threshold for divergence in the Mandelbrot set (default is 2.0).
        max_iter (int): Maximum number of iterations for checking divergence.

    Returns:
        np.ndarray: Array of estimated area of the Mandelbrot set for each iteration.
    """
    area = (BOUNDING_BOX_X[1] - BOUNDING_BOX_X[0]) * (BOUNDING_BOX_Y[1] - BOUNDING_BOX_Y[0])
    iterations = i
    size_iterations = np.zeros(iterations)

    for j in range(iterations):
        # Generate random points within the bounding rectangle
        x_coordinates = np.random.uniform(BOUNDING_BOX_X[0], BOUNDING_BOX_X[1], sample_size)
        y_coordinates = np.random.uniform(BOUNDING_BOX_Y[0], BOUNDING_BOX_Y[1], sample_size)

        # Check if the points are in the Mandelbrot set using the vectorized function
        mask = _check_points_mandelbrot(x_coordinates, y_coordinates)

        # Calculate the area estimation for this iteration
        points_in_set = np.count_nonzero(mask)  
        area_estimate = (points_in_set / sample_size) * area  

        # Store the result of this iteration
        size_iterations[j] = area_estimate
    
    return size_iterations


In [ ]:
def _calculate_iteration_statistics(size_iterations: np.ndarray) -> Tuple[float, float]:
    """
    Calculate the mean and standard deviation of the given size iterations array.

    Parameters:
        size_iterations (np.ndarray): Array of area estimates from each Monte Carlo iteration.

    Returns:
        Tuple[float, float]: A tuple containing the mean and standard deviation.
    """
    mean = np.mean(size_iterations)
    std_dev = np.std(size_iterations)
    return mean, std_dev

In [ ]:
#  Testing the function
monte_carlo_random(10000,1000)

### 2.1: Convergence of $A_{i,s} \rightarrow A_M$ 

We set the sample size to $s=1000$, and investigate the convergence of the area when changing the number of iterations for all $j<i$ such that $i = 3000$ and $j = \{100k: k \in \{1, 2, ..., 30\}\}$

In [ ]:
def monte_carlo_trajectory_iterations(
    sample_size: int, 
    start_iter: int, 
    end_iter: int, 
    measurements_amount: int,
    monte_carlo_method: Callable[[int, int], np.ndarray],
) -> Tuple[np.ndarray, np.ndarray]:
    """
    Perform a Monte Carlo convergence test to estimate the Mandelbrot set area and
    standard deviation over varying numbers of iterations.

    Parameters:
        sample_size (int): Number of samples per iteration.
        start_iter (int): Starting number of iterations.
        end_iter (int): Ending number of iterations.
        measurements_amount (int): Total number of different iteration counts to test.
        monte_carlo_method (Callable): Monte Carlo method function to use for estimation.
            The function should accept two arguments (iterations, sample_size) and return
            an array of area estimates.

    Returns:
        Tuple[np.ndarray, np.ndarray]: A tuple containing:
            - estimation_per_iteration (np.ndarray): Array of estimated areas per iteration count.
            - std_dev_per_iteration (np.ndarray): Array of standard deviations per iteration count.
    """
    # Generate an array of iteration counts between start_iter and end_iter
    number_of_iterations = np.linspace(start_iter, end_iter, measurements_amount, dtype=int)
    estimation_per_iteration = np.zeros(len(number_of_iterations))  # Store estimated area
    std_dev_per_iteration = np.zeros(len(number_of_iterations))     # Store standard deviation

    # Define a function for parallelized computation of area estimates per iteration
    def compute_estimate(iterations):
        area_estimates = monte_carlo_method(iterations, sample_size)
        estimation, std_dev = _calculate_iteration_statistics(area_estimates)
        return estimation, std_dev

    # Parallelize across iteration counts
    results = Parallel(n_jobs=measurements_amount)(
        delayed(compute_estimate)(iterations) for iterations in number_of_iterations
    )

    # Store results into respective arrays
    for i, (estimation, std_dev) in enumerate(results):
        estimation_per_iteration[i] = estimation
        std_dev_per_iteration[i] = std_dev

    return estimation_per_iteration, std_dev_per_iteration

In [ ]:
def monte_carlo_trajectory_samples(
    iteration: int, 
    start_sample: int, 
    end_sample: int, 
    measurements_amount: int,
    monte_carlo_method: Callable[[int, int], np.ndarray],
) -> Tuple[np.ndarray, np.ndarray]:
    """
    Perform a Monte Carlo convergence test to estimate the Mandelbrot set area and
    standard deviation over varying numbers of iterations.

    Parameters:
        sample_size (int): Number of samples per iteration.
        start_iter (int): Starting number of iterations.
        end_iter (int): Ending number of iterations.
        measurements_amount (int): Total number of different iteration counts to test.
        monte_carlo_method (Callable): Monte Carlo method function to use for estimation.
            The function should accept two arguments (iterations, sample_size) and return
            an array of area estimates.

    Returns:
        Tuple[np.ndarray, np.ndarray]: A tuple containing:
            - estimation_per_iteration (np.ndarray): Array of estimated areas per iteration count.
            - std_dev_per_iteration (np.ndarray): Array of standard deviations per iteration count.
    """
    # Generate an array of iteration counts between start_iter and end_iter
    number_of_samples = np.linspace(start_sample, end_sample, measurements_amount, dtype=int)
    estimation_per_sample = np.zeros(len(number_of_samples))  # Store estimated area
    std_dev_per_iteration = np.zeros(len(number_of_samples))     # Store standard deviation

    # Define a function for parallelized computation of area estimates per iteration
    def compute_estimate(sample):
        area_estimates = monte_carlo_method(iteration, sample)
        estimation, std_dev = _calculate_iteration_statistics(area_estimates)
        return estimation, std_dev

    # Parallelize across iteration counts
    results = Parallel(n_jobs=measurements_amount)(
        delayed(compute_estimate)(sample) for sample in number_of_samples
    )

    # Store results into respective arrays
    for i, (estimation, std_dev) in enumerate(results):
        estimation_per_sample[i] = estimation
        std_dev_per_iteration[i] = std_dev

    return std_dev_per_iteration, std_dev_per_iteration

In [ ]:
def plot_monte_carlo_methods(
    method_names: List[str], 
    method_results: List[Tuple[np.ndarray, np.ndarray]],
    start_iter: int,
    end_iter: int,
    measurements_amount: int,
    expected_value: float,
    plot_std: bool = False
) -> None:
    """
    Plot Monte Carlo convergence for different methods, showing both the estimated area, 
    the standard deviation around it (optional), and the difference in estimated area for each method.

    Parameters:
        method_names (List[str]): List of method names for labeling the plot.
        method_results (List[Tuple[np.ndarray, np.ndarray]]): List of tuples containing 
            the estimation and standard deviation arrays for each method.
        start_iter (int): Starting number of iterations.
        end_iter (int): Ending number of iterations.
        measurements_amount (int): Total number of different iteration counts to test.
        expected_value (float): The "literary" or expected value to compare against.
        plot_std (bool): Whether to plot standard deviation as a shaded region (default is False).

    Returns:
        None: This function plots the results directly.
    """
    # Generate an array of iteration counts between start_iter and end_iter
    number_of_iterations = np.linspace(start_iter, end_iter, measurements_amount, dtype=int)

    # Set up the figure and subplots
    figure, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))

    # Loop over each method result and plot
    for method_name, (estimation, std_dev) in zip(method_names, method_results):
        # Plot the estimated area on the first subplot
        ax1.plot(number_of_iterations, estimation, label=method_name)
        
        # Optionally plot the standard deviation as a shaded region around the estimated area
        if plot_std:
            ax1.fill_between(
                number_of_iterations,
                estimation - std_dev,
                estimation + std_dev,
                alpha=0.2,
                label=f"{method_name} ± 1 Std Dev"
            )
        
        # Calculate the difference from the final estimated area (convergence reference)
        final_estimation = estimation[-1]  # Use the last estimation as the reference
        difference = np.abs(estimation[:-1] - final_estimation)  # Difference with the reference
        
        # Plot the difference in estimated area on the second subplot
        ax2.plot(number_of_iterations[:-1], difference, label=method_name)

    # Configure the first subplot (Estimated Area)
    ax1.set_xlabel("Number of Iterations")
    ax1.set_ylabel("Estimated Area of Mandelbrot Set")
    ax1.set_title("Estimated Area of Mandelbrot Set with Standard Deviation" if plot_std else "Estimated Area of Mandelbrot Set")
    ax1.grid(True)
    
    # Plot the expected (literary) value as a horizontal line
    ax1.axhline(y=expected_value, color='r', linestyle='--', label=f'Literary Value = {expected_value}')
    ax1.legend(title="Methods")

    # Configure the second subplot (Difference in Estimated Area)
    ax2.set_xlabel("Number of Iterations")
    ax2.set_ylabel("Difference relative to final estimated area")
    ax2.set_title("Difference in Estimated Area Convergence")
    ax2.grid(True)
    ax2.legend(title="Methods")

    # Display the plots with tight layout
    plt.tight_layout()
    plt.show()

In [ ]:
def plot_monte_carlo_convergence_difference(
    method_names: List[str], 
    method_results: List[Tuple[np.ndarray, np.ndarray]],
    start_iter: int,
    end_iter: int,
    total_iterations: int,
    expected_value: float,
    plot_std: bool = False
) -> None:
    """
    Plot Monte Carlo convergence difference relative to the expected (literary) value for each iteration,
    with standard deviation shown as a shaded region if desired.

    Parameters:
        method_names (List[str]): List of method names for labeling the plot.
        method_results (List[Tuple[np.ndarray, np.ndarray]]): List of tuples containing 
            the estimation and standard deviation arrays for each method.
        start_iter (int): Starting number of iterations.
        end_iter (int): Ending number of iterations.
        total_iterations (int): Total number of different iteration counts to test.
        expected_value (float): The "literary" or expected value to compare against.
        plot_std (bool): Whether to plot standard deviation as a shaded region (default is False).
        
    Returns:
        None: This function plots the results directly.
    """
    # Generate an array of iteration counts between start_iter and end_iter
    number_of_iterations = np.linspace(start_iter, end_iter, total_iterations, dtype=int)

    # Set up the figure
    plt.figure(figsize=(10, 6))

    # Loop over each method result and plot
    for method_name, (estimation, std_dev) in zip(method_names, method_results):
        # Calculate the difference from the expected (literary) value
        difference = np.abs(estimation - expected_value)

        # Plot the convergence difference relative to the expected (literary) value
        plt.plot(number_of_iterations, difference, label=method_name)
        
        # Optionally plot the standard deviation as a shaded region around the difference
        if plot_std:
            plt.fill_between(
                number_of_iterations,
                difference - std_dev,
                difference + std_dev,
                alpha=0.2,
                label=f"{method_name} ± 1 Std Dev"
            )

    # Configure the plot
    plt.xlabel("Number of Iterations")
    plt.ylabel("Difference from Expected Value")
    plt.title("Convergence Difference Relative to Expected Value" + (" with Standard Deviation" if plot_std else ""))
    plt.legend(title="Methods")
    plt.grid(True)

    # Display the plot
    plt.tight_layout()
    plt.show()


In [ ]:
def plot_monte_carlo_trajectory_samples(
    method_names: List[str], 
    method_results: List[Tuple[np.ndarray, np.ndarray]],
    start_sample: int,
    end_sample: int,
    measurements_amount: int,
    expected_value: float,
    plot_std: bool = False
) -> None:
    """
    Plot Monte Carlo convergence difference relative to the expected (literary) value for each iteration,
    with standard deviation shown as a shaded region if desired.

    Parameters:
        method_names (List[str]): List of method names for labeling the plot.
        method_results (List[Tuple[np.ndarray, np.ndarray]]): List of tuples containing 
            the estimation and standard deviation arrays for each method.
        start_iter (int): Starting number of iterations.
        end_iter (int): Ending number of iterations.
        measurements_amount (int): Total number of different iteration counts to test.
        expected_value (float): The "literary" or expected value to compare against.
        plot_std (bool): Whether to plot standard deviation as a shaded region (default is False).
        
    Returns:
        None: This function plots the results directly.
    """
    # Generate an array of iteration counts between start_iter and end_iter
    number_of_samples = np.linspace(start_sample, end_sample, measurements_amount, dtype=int)

    # Set up the figure
    plt.figure(figsize=(10, 6))

    # Loop over each method result and plot
    for method_name, (estimation, std_dev) in zip(method_names, method_results):
        # Calculate the difference from the expected (literary) value
        difference = np.abs(estimation - expected_value)

        # Plot the convergence difference relative to the expected (literary) value
        plt.plot(number_of_samples, difference, label=method_name)
        
        # Optionally plot the standard deviation as a shaded region around the difference
        if plot_std:
            plt.fill_between(
                number_of_samples,
                difference - std_dev,
                difference + std_dev,
                alpha=0.2,
                label=f"{method_name} ± 1 Std Dev"
            )

    # Configure the plot
    plt.xlabel("Number of Iterations")
    plt.ylabel("Difference from Expected Value")
    plt.title("Convergence Difference Relative to Expected Value" + (" with Standard Deviation" if plot_std else ""))
    plt.legend(title="Methods")
    plt.grid(True)

    # Display the plot
    plt.tight_layout()
    plt.show()

In [ ]:
# Define parameters
sample_size = 1000
start_iter = 100
end_iter = 1000000
measurements_amount = 10

# Define method names and results
methods = [monte_carlo_random]  # Replace with actual functions
method_names = ['Random Sampling']

# Calculate results for each method
results = [monte_carlo_trajectory_iterations(sample_size, start_iter, end_iter, measurements_amount, method) for method in methods]

In [ ]:
# Plotting
plot_monte_carlo_methods(method_names, results, start_iter, end_iter, measurements_amount, expected_value=LITERARY_VALUE, plot_std=False)

# plot_monte_carlo_convergence_difference(method_names, results, start_iter, end_iter, measurements_amount, expected_value=LITERARY_VALUE, plot_std=False)

## Problem 3: Different Sampling Methods

Explore different sampling methods. Namely:
1. Pure random sampling
2. Latin Hypercube sampling
3. Orthogonal sampling

### 3.1: Pure Random

In [ ]:
# TODO

### 3.2: Latin Hypercube

In [ ]:
# TODO

### 3.3: Orthogonal

In [ ]:
from scipy.stats import qmc

def oa_lhs(p, d, seed=None):
    """Generates an orthogonal array Latin hypercube sample.

    Author: Pamphile Tupui ROY
    Source: https://gist.github.com/tupui/3b79ecea1631e8925f3d47069d435b0f

    Inputs:
        p: number of levels
        d: number of dimensions
        seed: random seed

    Output: orthogonal array Latin hypercube sample
    """
    oa_sample = np.zeros(shape=(p**2, p + 1))

    arrays = np.tile(np.arange(p), (2, 1))
    oa_sample[:, :2] = np.stack(np.meshgrid(*arrays), axis=-1).reshape(-1, 2)
    for p_ in range(1, p):
        oa_sample[:, 2 + p_ - 1] = np.mod(oa_sample[:, 0] + p_ * oa_sample[:, 1], p)

    # scramble the OA
    oa_sample_ = np.empty(shape=(p**2, p + 1))
    for j in range(p + 1):
        perms = np.random.permutation(p)
        for k in range(p):
            idx = np.where(oa_sample[:, j] == k)[0]
            oa_sample_[idx, j] = perms[k]
    
    oa_sample = oa_sample_

    # Convert to OA-LHS
    oa_lhs_sample = np.zeros(shape=(p**2, p + 1))
    for j in range(p + 1):
        for k in range(p):
            idx = np.where(oa_sample[:, j] == k)[0]
            lhs = qmc.LatinHypercube(d=1, centered=True, seed=seed).random(p).flatten()
            oa_lhs_sample[:, j][idx] = lhs + oa_sample[:, j][idx]

    oa_lhs_sample /= p

    if d is not None:
        oa_lhs_sample = oa_lhs_sample[:, :d]
    return oa_lhs_sample

In [ ]:
def monte_carlo_oa_lhs(iterations, sample_size, x_min=-2, x_max=1.5, y_min=-2, y_max=2, seed=None):
    # Calculate p to ensure p^2 >= sample_size
    p = int(np.ceil(np.sqrt(sample_size)))
    size_iterations = np.zeros(iterations)

    for j in range(iterations):
        point_in = np.zeros(sample_size)    # Storing points inside the Mandelbrot set
        
        # Generate OA-LHS samples for the bounding rectangle
        oa_lhs_samples = oa_lhs(p, 2, seed=seed)
        
        # Scale the OA-LHS samples to the desired region
        oa_lhs_samples[:, 0] = oa_lhs_samples[:, 0] * (x_max - x_min) + x_min
        oa_lhs_samples[:, 1] = oa_lhs_samples[:, 1] * (y_max - y_min) + y_min

        # Limit the samples to sample_size
        oa_lhs_samples = oa_lhs_samples[:sample_size]
        
        for i in range(sample_size):
            x_cordinate, y_cordinate = oa_lhs_samples[i]
            if check_mandelbrot(x_cordinate, y_cordinate) == 1:
                point_in[i] = 1
        
        # Calculate the estimated area for this iteration
        size = (np.count_nonzero(point_in == 1) / sample_size) * ((x_max - x_min) * (y_max - y_min))
        size_iterations[j] = size   
    
    # Calculate the average estimated area across all iterations
    average_size = np.mean(size_iterations)
    
    return average_size

In [ ]:
# Example usage
average_size = monte_carlo_oa_lhs(iterations=100, sample_size=10000, seed=42)
print(f"Estimated area of the Mandelbrot set: {average_size}")

## Problem 4: Improvements to Monte Carlo Convergence Rate

Formulate and test a method to further improve the convergence rate of the Monte
Carlo approach.